In [25]:
#Name: Ishita Bansal
#Roll No: 101903132
#Batch: 3COE13
#Email: ibansal1_be19@thapar.edu

In [26]:
import cudf as cd
import cupy as np

In [27]:
dftest = cd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
dftest.head()
dftrain = cd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
dftrain.head()

In [28]:
dftrain.columns

In [29]:
dftrain.dtypes

In [30]:
dftrain.iloc[:,1:81].describe()

In [31]:
dftrain.isnull().sum().sort_values(ascending=False)

In [32]:
def cleaning_data_none(dftrain: dict, fields: dict):
    for field in fields:
        dftrain[field].fillna('None', inplace=True)

def cleaning_data_int(dftrain: dict, fields: dict):
    for field in fields:
        dftrain[field].fillna(0, inplace=True)

def cleaning_data_median(dftrain: dict, fields: dict):
    for field in fields:
        dftrain[field].fillna(dftrain[field].median(), inplace=True)

In [33]:
fields_clean_none = ['PoolQC',
                     'Alley',
                     'FireplaceQu',
                     'MasVnrType',
                     'Electrical',
                     'BsmtFinType2',
                     'BsmtFinType1',
                     'BsmtExposure',
                     'BsmtQual',
                     'BsmtCond',
                     'Fence',
                     'MiscFeature',
                     'GarageCond',
                     'GarageQual',
                     'GarageFinish',
                     'GarageType',
                     'SaleType',
                     'Utilities',
                     'Exterior1st',
                     'Exterior2nd',
                     'KitchenQual',
                     'Functional']

fields_clean_int = ['GarageYrBlt', 'MSZoning', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath']

fields_clean_median = ['LotFrontage',
                        'MasVnrArea',
                        'BsmtUnfSF',
                        'TotalBsmtSF',
                         'GarageCars',
                         'GarageArea']
                       

cleaning_data_none(dftrain, fields_clean_none)
cleaning_data_int(dftrain, fields_clean_int)
cleaning_data_median(dftrain, fields_clean_median)

In [34]:
features=dftrain.columns
features=list(features[1:len(features)-1])

In [35]:
len(features)

In [36]:
features

In [37]:
dftrain.head()

In [38]:
import pandas as pd
df_types = pd.DataFrame(dftrain.dtypes, columns=["types"])
df_types_object = df_types[df_types["types"] == "object"]
for field_obj in df_types_object.index:
    dftrain[field_obj] = dftrain[field_obj].astype('category').cat.codes
dftrain.head()

In [39]:
dftrain['SalePrice'] = dftrain['SalePrice'].astype('float64')

In [40]:
Y = dftrain['SalePrice']
Y = np.array(Y)
Y.reshape(-1,1)
Y

In [41]:
X = dftrain[features]
X

In [42]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_scaled = scaler.fit_transform(X.as_matrix())

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_scaled, Y, test_size = 0.3, random_state = 42)

In [44]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

In [45]:
import cuml
from cuml import LinearRegression
X_test

In [46]:
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'eig')
reg = lr.fit(X_train, Y_train)
print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)
print("Predictions: ")
pred = lr.predict(X_test)
print(pred)
print("MSE: ")
print(cuml.metrics.regression.mean_squared_error(Y_test,pred))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(Y_test,pred))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(Y_test,pred))

In [47]:
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']

In [48]:
for i in algorithm:
    print("Algorithm:")
    print(i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    reg = lr.fit(X_train,Y_train)
    preds = lr.predict(X_test)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(Y_test,preds))
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(Y_test,preds))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(Y_test,preds))

In [49]:
import cuml
from cuml import Ridge
alpha = np.array([1e-5])
ridge_eig = Ridge(alpha = alpha, fit_intercept = True, normalize = False, solver= "eig")

In [50]:
result_ridge = ridge_eig.fit(X_train, Y_train)
print("Coefficients: ")
print(result_ridge.coef_)
print("Intercept: ")
print(result_ridge.intercept_)

In [51]:
preds = result_ridge.predict(X_test)

In [52]:
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(Y_test,preds))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(Y_test,preds))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(Y_test,preds))

In [53]:
alpha = np.array([1e-5])
ridge_svd = Ridge(alpha = alpha, fit_intercept = True, normalize = False, solver= "svd")

In [54]:
result_ridge = ridge_svd.fit(X_train, Y_train)
print("Coefficients: ")
print(result_ridge.coef_)
print("Intercept: ")
print(result_ridge.intercept_)

In [60]:
preds = result_ridge.predict(X_test)

In [61]:
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(Y_test,preds))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(Y_test,preds))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(Y_test,preds))

In [62]:
from cuml.linear_model import Lasso
ls = Lasso(alpha = 0.1)

In [63]:
result_lasso = ls.fit(X_train, Y_train)
print("Coefficients:")
print(result_lasso.coef_)
print("intercept:")
print(result_lasso.intercept_)

In [64]:
preds = result_lasso.predict(X_test)

In [65]:
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(Y_test,preds))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(Y_test,preds))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(Y_test,preds))